In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
import json
import sys
import time

In [2]:
def patch_pyspark_accumulators():
    from inspect import getsource
    import pyspark.accumulators as pa
    exec(getsource(pa._start_update_server).replace("localhost", "127.0.0.1"), pa.__dict__)


if __name__ == '__main__':
    #patch_pyspark_accumulators()
    start = time.time()
    business_path = './yelp_dataset/business.json'
    user_path = './yelp_dataset/user.json'
    review_path = './yelp_dataset/review.json'
    #sc.stop()
    conf = SparkConf()
    sc = SparkContext(appName='mm_exp', conf=conf)
    
    BusinessRDD = sc.textFile(business_path).map(lambda x: json.loads(x))
    print('#all restaurants: ', BusinessRDD.count())
    def countByState(data):
        cnt = len(list(data))
        return cnt
    # 打印各个state的店铺数量
    res = BusinessRDD.groupBy(lambda x: x['state']).mapValues(countByState).collect()
    print(res)
    # 选择店铺最多的TN，储存到selected_data中
    BusinessRDD = BusinessRDD \
                        .filter(lambda x: x['state'] == 'TN') \
                        .filter(lambda x: x['review_count'] >= 10) \
                        .map(lambda x: (x['business_id'], x['stars'], x['review_count'], x['attributes'], x['categories']))
    print('#selected restaurants: ', BusinessRDD.count())
    BusinessStats = BusinessRDD.collect()
    BusinessOutput = {item[0]:{'stars':item[1], 'review_count':item[2], 'attributes':item[3], 'categories':item[4]} for item in BusinessStats}
    with open('./selected_data/business.json', 'w') as f:
        json.dump(BusinessOutput, f)
    
    # 储存user中有效的信息到selected_data中
    UserRDD = sc.textFile(user_path).map(lambda x: json.loads(x))
    UserRDD = UserRDD \
                    .filter(lambda x: x['review_count'] >= 10) \
                    .map(lambda x: (x['user_id'], x['average_stars'], x['review_count']))
    print('#selected users: ', UserRDD.count())
    UserStats = UserRDD.collect()
    UserOutput = {item[0]:{'average_stars':item[1], 'review_count':item[2]} for item in UserStats}
    with open('./selected_data/user.json', 'w') as f:
        json.dump(UserOutput, f)
   
    sc.stop()
    end = time.time()
    print('time: ' + str(end-start))

22/05/28 16:37:41 WARN Utils: Your hostname, Lyna.local resolves to a loopback address: 127.0.0.1; using 10.20.83.130 instead (on interface en0)
22/05/28 16:37:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/28 16:37:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


#all restaurants:  150346


[('TN', 12056), ('DE', 2265), ('AZ', 9912), ('PA', 34039), ('IN', 11247), ('AB', 5573), ('NV', 7715), ('ID', 4467), ('IL', 2145), ('MT', 1), ('CA', 5203), ('MO', 10913), ('LA', 9924), ('NC', 1), ('CO', 3), ('HI', 2), ('TX', 4), ('SD', 1), ('FL', 26330), ('NJ', 8536), ('WA', 2), ('UT', 1), ('MI', 1), ('XMS', 1), ('MA', 2), ('VI', 1), ('VT', 1)]
#selected restaurants:  8209


#selected users:  726519


time: 30.39393925666809
